<a href="https://colab.research.google.com/github/itslisaram/UEM-IA/blob/main/Lo_Fi_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inteligencia Artificial - Generar Musica Lo-Fi mediante IA

Este proyecto utiliza redes neuronales LSTM con el objetivo de generar música lo-fi a traves de archivos MIDI. El proceso incluye tanto la carga y preprocesamiento de los archivos MIDI como el entranamiento
de un modelo LSTM para predecir secuencias de notas y generar nuevas composiciones

Los pasos que he seguido en el proyecto son:
  1. Carga de datos: se leen archivos MIDI
  2. Preprocesamiento: conversion de MIDI a secuencias de notas y normalizacion
  3. Entrenamiento: el modelo LSTM aprende patrones musicales en las sencias de notas
  4. Generacion: creacion de nuevas secuencias MIDI basadas en lo que el modelo ha aprendido
  5. Conversion a WAV: se utiliza FluidSynth para transformar MIDI a WAV
  6. Descarga resultado: descarga del archivo WAV para su reproduccion

Herramientas utilizadas:
  -TensorFlow/Keras: para las red neuronal LSTM
  -Mido: para la manipulacion de archivos MIDI
  -FluidSynth: para la conversion de MIDI a WAV

Los archivos MIDI utilizados para entrenar el modelo proceden de: https://www.kaggle.com/datasets/zakarii/lofi-hip-hop-midi/data

In [ ]:
# Instalar dependencias
!apt-get install fluidsynth
!pip install mido pretty_midi tensorflow numpy


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libevdev2 libfluidsynth3 libgudev-1.0-0 libinput-bin libinput10
  libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a libqt5dbus5 libqt5gui5 libqt5network5
  libqt5svg5 libqt5widgets5 libwacom-bin libwacom-common libwacom9 libxcb-icccm4 libxcb-image0
  libxcb-keysyms1 libxcb-render-util0 libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1
  libxkbcommon-x11-0 qsynth qt5-gtk-platformtheme qttranslations5-l10n timgm6mb-soundfont
Suggested packages:
  fluid-soundfont-gs qt5-image-formats-plugins qtwayland5 jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libevdev2 libfluidsynth3 libgudev-1.0-0 libinput-bin libinput10
  libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a libqt5dbus5 libqt5gui5 libqt5network5
  libqt5svg5 libqt5widgets5 libwacom-bin libwacom-common libwacom9 libxcb-icc

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import mido
from mido import MidiFile, MidiTrack, Message
import random
import subprocess

In [ ]:
# Directorio donde se encuentran los MIDI que utilizaremos
data_dir = '/content/MIDI'

In [ ]:
# Comprobamos si Colab tiene Soundfont -> Sirve para ajustar la conversion MIDI a WAV
!ls /usr/share/sounds/sf2
soundfont = "/usr/share/sounds/sf2/FluidR3_GM.sf2"

default-GM.sf2	FluidR3_GM.sf2	TimGM6mb.sf2


In [ ]:
# Paso 1: Leer los archivos MIDI y convertirlos en notas
def midi_to_notes(midi_file):
    midi = MidiFile(midi_file)
    notes = []
    for msg in midi:
        if msg.type == "note_on" and msg.velocity > 0:
            notes.append(msg.note)
    return notes

In [ ]:
# Leer las notas de todos los archivos MIDI
data = []
for file in os.listdir(data_dir):
    if file.endswith(".mid"):
        file_path = os.path.join(data_dir, file)
        data.extend(midi_to_notes(file_path))

In [ ]:
# Paso 2: Se crean las secuencias de entrenamiento
seq_length = 50  # Tamaño de secuencia para el modelo
X, y = [], []
for i in range(len(data) - seq_length):
    X.append(data[i:i+seq_length])
    y.append(data[i+seq_length])

X = np.array(X)
y = np.array(y)

In [ ]:
# Paso 3: Normalizacion de los datos
X = X / 127.0  # MIDI usa valores de nota de 0 a 127
y = y / 127.0

In [ ]:
# Paso 4: Se contruye el modelo LSTM
model = Sequential([
    tf.keras.layers.Input(shape=(seq_length, 1)),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(128),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dense(1, activation='linear')
])

model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))

In [ ]:
# Paso 5: Entrenamos el modelo
epochs = 50
model.fit(X, y, epochs=epochs, batch_size=64)

Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 18s 212ms/step - loss: 0.0539
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 22s 233ms/step - loss: 0.0080
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 12s 193ms/step - loss: 0.0078
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 22s 215ms/step - loss: 0.0079
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 21s 216ms/step - loss: 0.0078
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 214ms/step - loss: 0.0079
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 13s 211ms/step - loss: 0.0073
Epoch 8/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 210ms/step - loss: 0.0078
Epoch 9/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 13s 200ms/step - loss: 0.0078
Epoch 10/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 21s 215ms/step - loss: 0.0074
Epoch 11/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 215ms/step - loss: 0.0073
Epoch 12/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 213ms/step - loss: 0.0075
Epoch 13/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 14s 226ms/step - loss: 0.0077
Epoch 14/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 19s 208ms/step - loss: 0.0075
Epoch 15/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 21s 

In [ ]:
# Paso 6: Generamos nueva musica
def generate_midi(model, seed_notes, length=100):
    generated = seed_notes.copy()
    input_seq = seed_notes.copy()
    for _ in range(length):
        input_seq = np.array(input_seq).reshape(1, seq_length, 1) / 127.0
        next_note = model.predict(input_seq)[0][0] * 127.0
        next_note = int(np.clip(next_note, 0, 127))
        generated.append(next_note)
        input_seq = generated[-seq_length:]
    return generated

# Semilla inicial (secuencia aleatoria del dataset)
seed_index = random.randint(0, len(data) - seq_length)
seed_notes = data[seed_index:seed_index + seq_length]
new_midi_notes = generate_midi(model, seed_notes)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━

In [ ]:
# Paso 7: La musica generada se guarda como MIDI
def notes_to_midi(notes, filename="generated_lofi.mid"):
    midi = MidiFile()
    track = MidiTrack()
    midi.tracks.append(track)
    for note in notes:
        track.append(Message("note_on", note=note, velocity=64, time=200))
        track.append(Message("note_off", note=note, velocity=64, time=200))
    midi.save(filename)

notes_to_midi(new_midi_notes)
print("Música generada y guardada en 'generated_lofi.mid'")

Música generada y guardada en 'generated_lofi.mid'


In [ ]:
# Paso 8: Se convierte el archivo de musica MIDI generado a WAV con FluidSynth
def midi_to_wav(midi_file, soundfont=soundfont, output_wav="generated_lofi.wav"):
    command = f"fluidsynth -ni {soundfont} {midi_file} -F {output_wav} -r 44100"
    subprocess.run(command, shell=True)
    print(f"Archivo WAV guardado en '{output_wav}'")

midi_to_wav("generated_lofi.mid")

Archivo WAV guardado en 'generated_lofi.wav'


In [ ]:
# Paso 9: Descarga el archivo WAV generado
def download_wav(file_path):
    from google.colab import files
    files.download(file_path)

print("Descargando archivo WAV...")
download_wav("generated_lofi.wav")

Descargando archivo WAV...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>